In [1]:
import os
import json
import pickle
import numpy as np
import pandas as pd
from tqdm import tqdm
import torch
from torch.utils.data import Dataset
from transformers import (
    AutoTokenizer,
    AutoConfig,
    AutoModelForSequenceClassification,
    AutoModelForSeq2SeqLM,
    Seq2SeqTrainer,
    Seq2SeqTrainingArguments,
    DataCollatorForSeq2Seq,
    TrainerCallback
)

In [2]:
EPOCHS = 1
LEARNING_RATE = 1e-4
BATCH_SIZE = 8
OUTPUT_PATH = "./output"
MODEL = "t5-base"
DATA_PATH = '../data/qrels.train.tsv'

In [3]:
# Converting MS-MARCO to torch dataset for fine-tuning


class dataset(Dataset):
    def __init__(self, data):
        self.data = data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        sample = self.data[idx]
        text = f'Query: {sample[0]} Document: {sample[1]} Relevant:'
        return {
          'text': text,
          'labels': sample[2],
        }

In [4]:
# Using T5-base for fine-tuning
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.manual_seed(101)

model = AutoModelForSeq2SeqLM.from_pretrained(MODEL).to(device)
tokenizer = AutoTokenizer.from_pretrained(MODEL)

In [5]:
# Processing MS-MARCO dataset

train_samples = []
with open(DATA_PATH, 'r', encoding='utf-8') as file:
    for num, line in tqdm(enumerate(file)):
        if num > 6.4e5 * 10:
            break
        query, positive, negative = line.split("\t")
        train_samples.append((query, positive, 'true'))
        train_samples.append((query, negative, 'false'))

0it [00:00, ?it/s]

1065522it [00:13, 77216.40it/s]


In [6]:
dataset_train = dataset(train_samples)

In [7]:
# Setting training arguments and parameters

train_args = Seq2SeqTrainingArguments(
    predict_with_generate=True,
    remove_unused_columns=False, 
    output_dir=OUTPUT_PATH,
    do_train=True,
    save_strategy='steps',
    save_steps=10000,
    logging_steps=100,
    per_device_train_batch_size=BATCH_SIZE,
    gradient_accumulation_steps=16,
    learning_rate=LEARNING_RATE,
    weight_decay=5e-5,
    num_train_epochs=EPOCHS,
    warmup_steps=1000,
    adafactor=True,
    seed=1,
    disable_tqdm=False,
    load_best_model_at_end=False,
    dataloader_pin_memory=False,
)

/home/pratham/miniconda3/envs/torch/lib/python3.12/site-packages/transformers/training_args.py:1725: FutureWarning: `--adafactor` is deprecated and will be removed in version 5 of 🤗 Transformers. Use `--optim adafactor` instead
  warnings.warn(


In [8]:
def data_collate(batch):
    texts = [example["text"] for example in batch]
    tokenized = tokenizer(
        texts,
        padding=True,
        truncation="longest_first",
        return_tensors="pt",
        max_length=512,
    )

    tokenized["labels"] = tokenizer(
        [example["labels"] for example in batch], return_tensors="pt"
    )["input_ids"]

    for name in tokenized:
        tokenized[name] = tokenized[name].to(device)

    return tokenized

In [9]:
trainer = Seq2SeqTrainer(
    model=model,
    args=train_args,
    train_dataset=dataset_train,
    tokenizer=tokenizer,
    data_collator=data_collate,
)

trainer.train()

trainer.save_model(OUTPUT_PATH)
trainer.save_state()

/tmp/ipykernel_30333/2194772448.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(
Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Step,Training Loss
100,4.570800
200,0.400900
300,0.295000
400,0.273000
500,0.263700
600,0.257800


KeyboardInterrupt: 